In [1]:
# LQ problem Optimal Growth Model 
# Adapted to Hansen Real Business Cycle model from  McCandless (2008)
# tested in Julia 1.5.0
# this code is part of chapter 4, "Dynamic Programming" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com
            
#calibration            
theta  =.36;
beta   =.99;
delta  =.025;
A      =1.72;
            
#steady state            
kbar   =12.6695;
hbar   =.3335;
ybar   =kbar^theta*hbar^(1-theta);
cbar   =ybar-delta*kbar;
aa     =(theta*ybar/kbar+1-delta);

#setup matrices
a=zeros(3,3);
a[1,1]=-1/(2*cbar*cbar)*aa*aa-1/(2*cbar)*theta*(1-theta)*ybar/(kbar*kbar);
a[1,2]=1/(2*cbar*cbar)*aa;
a[2,1]=1/(2*cbar*cbar)*aa;
a[1,3]=-1/(2*cbar*cbar)*aa*(1-theta)*ybar/hbar;
a[1,3]=a[1,3]+1/(2*cbar)*theta*(1-theta)*ybar/(kbar*hbar);
a[3,1]=a[1,3];
a[2,2]=-1/(2*cbar*cbar);
a[2,3]=1/(2*cbar*cbar)*(1-theta)*ybar/hbar;
a[3,2]=a[2,3];
a[3,3]=-1/(2*cbar*cbar)*(1-theta)*ybar/hbar*(1-theta)*ybar/hbar;
a[3,3]=a[3,3]-1/(2*cbar)*theta*(1-theta)*ybar/(hbar*hbar);
a[3,3]=a[3,3]-A/(2*(1-hbar)*(1-hbar));


In [2]:
x=[kbar kbar hbar]';

m=zeros(4,4)
m[1,1]=log(kbar^theta*hbar^(1-theta)-delta*kbar)+A*log(1-hbar);
mm1=1/cbar*(theta*ybar/kbar+(1-delta));
mm2=(1-theta)*ybar/(cbar*hbar)-A/(1-hbar);
m[1,1]=m[1,1]-mm1*kbar+kbar/cbar-mm2*hbar;
e=m[1,1].+(x')*a*x;
m[1,1]=e[1]
e=(mm1/2).-transpose(a[1:3,1])*x;
m[1,2]=e[1];
m[2,1]=m[1,2];
e=(-1/(2*cbar)).-transpose(a[1:3,2])*x;
m[1,3]=e[1]
m[3,1]=m[1,3];
e=(mm2/2).-transpose(a[1:3,3])*x;
m[1,4]=e[1];
m[4,1]=m[1,4];
m[2:4,2:4]=a;

m

4×4 Array{Float64,2}:
 -1.63745   1.09965   -1.08865    1.93608
  1.09965  -0.605576   0.598565  -1.38229
 -1.08865   0.598565  -0.592579   1.40477
  1.93608  -1.38229    1.40477   -6.65903

In [3]:
AA=[1 0; 0 0];
B=[0 0; 1 0];
R=m[1:2,1:2];
Q=m[3:4,3:4];
W=m[1:2,3:4]';
P=[1 0 ;0 1];


In [4]:
inv(Q+beta*B'*P*B)

2×2 Array{Float64,2}:
 1.44141    0.304075
 0.304075  -0.0860252

In [5]:
#Iteration
zinv=zeros(2,2)
for i=1:1000
    zinv=inv(Q+beta*B'*P*B);
    z2=beta*AA'*P*B+W';
    P=R+beta*AA'*P*AA-z2*zinv*z2'
end
println("The matrix P is:", P)
F=-zinv*(W+beta*B'*P*AA)
println("The matrix F is:", F)

The matrix P is:[-96.36153981420405 0.8779195537357187; 0.877919553735719 -0.025897176284549905]
The matrix F is:[0.5869437166931122 0.9536736837231349; 0.4145647803411402 -0.006397551231004384]
